In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

import sklearn
sklearn.set_config(transform_output="pandas")

In [3]:
dataset = pd.read_csv("train.csv")

In [4]:
def cleaning(df):

    threshold = 100

    cat_feats = ["cap-shape","cap-surface","cap-color","does-bruise-or-bleed","gill-attachment",
                 "gill-spacing","gill-color","stem-root","stem-surface","stem-color","veil-type",
                 "veil-color","has-ring","ring-type","spore-print-color","spore-print-color",
                 "habitat","season"]

    for feat in cat_feats:
        df[feat] = df[feat].fillna('missing')
        #df.loc[df[feat].value_counts(dropna=False)[df[feat]].values < threshold, feat] = "noise"
        df[feat] = df[feat].astype('category')

    return df

dataset  = cleaning(dataset)

In [5]:
dataset

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,missing,missing,w,missing,missing,f,f,missing,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,missing,y,o,missing,missing,t,z,missing,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,missing,s,n,missing,missing,f,f,missing,l,w
3,3,e,3.88,f,y,g,f,s,missing,g,...,missing,missing,w,missing,missing,f,f,missing,d,u
4,4,e,5.85,x,l,w,f,d,missing,w,...,missing,missing,w,missing,missing,f,f,missing,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,missing,n,t,missing,missing,w,...,b,missing,w,u,w,t,g,missing,d,u
3116941,3116941,e,10.88,s,missing,w,t,d,c,p,...,missing,missing,w,missing,missing,f,f,missing,d,u
3116942,3116942,p,7.82,x,e,e,f,a,missing,w,...,missing,missing,y,missing,w,t,z,missing,d,a
3116943,3116943,e,9.45,p,i,n,t,e,missing,p,...,missing,y,w,missing,missing,t,p,missing,d,u


In [6]:
X = dataset
X = X.drop(["class"], axis="columns")
y = dataset["class"]

In [7]:
%%time

from lightgbm import LGBMClassifier

classifier = LGBMClassifier(n_estimators=500,
                            random_state=42,
                            max_bin=128,
                            colsample_bytree=0.6,
                            reg_lambda = 80,
                            learning_rate= 0.1, # Higher LR => Quicker?
                            device = 'gpu',
                            #num_threads=24,
                            verbose = -1).fit(X, y)

CPU times: user 11min, sys: 24.7 s, total: 11min 24s
Wall time: 28.3 s


In [8]:
test = pd.read_csv("test.csv")
test  = cleaning(test)

X_test = test

In [9]:
#X_test = X_test.drop(["class"], axis="1")
#y_test = test["class"]

y_pred = classifier.predict(X_test)

#from sklearn.metrics import matthews_corrcoef
#print(round(matthews_corrcoef(y_test, y_pred),5))

In [10]:
y_pred_df = pd.DataFrame(y_pred)

In [11]:
y_pred_df.insert(0,'id',test['id'])

In [12]:
y_pred_df

,id,0
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [13]:
y_pred_df.to_csv("predict_kaggle.csv", index=False)

In [31]:
y_test_110 = pd.read_csv('y_test_110_40_1.csv')

In [32]:
y_test_110

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,p
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,p


In [34]:
y_test_110_np = np.array(y_test_110['class'])

In [35]:
from sklearn.metrics import matthews_corrcoef
print(round(matthews_corrcoef(y_pred, y_test_110_np),5))

0.08915


In [37]:
print(f"Random Forest accuracy train - test: {accuracy_score(y_pred, y_test_110_np):.8f}")

Random Forest accuracy train - test: 0.56239136
